### **DAY 3 (11/01/26) – PySpark Transformations Deep Dive**

### Learn:

- PySpark vs Pandas comparison
- Joins (inner, left, right, outer)
- Window functions (running totals, rankings)
- User-Defined Functions (UDFs)

### 🛠️ Tasks:

1. Load full e-commerce dataset
2. Perform complex joins
3. Calculate running totals with window functions
4. Create derived features

In [0]:
df = spark.read.csv('/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv',header=True,inferSchema=True)
df.limit(5).display()

In [0]:
df1 = spark.read.csv('/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv',header=True,inferSchema=True)
df1.limit(5).display()

In [0]:
full_df = df.unionAll(df1)
full_df.count()

In [0]:
df = df.repartition("user_id")
df1 = df1.repartition("user_id")

In [0]:
full_df=full_df.repartition("user_id")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [0]:
repeated_customers = full_df\
    .withColumn("Month",F.month(F.col("event_time")))\
    .select('user_id','Month','event_type','price')\
    .groupby('user_id','Month')\
    .pivot("event_type", [ "view", "cart","purchase"])\
    .agg(F.count("user_id").alias('Count'))\
    .fillna(0)\
    .orderBy(F.asc('user_id'),F.asc('Month'))

windows=Window.partitionBy('user_id')
result = repeated_customers\
    .withColumn('Frequency',F.count("*").over(windows))\
    .filter(F.col('Frequency')>1)\
    .select('user_id','Month','view','cart','purchase')
result.display()


In [0]:
result_df=full_df.withColumn(
    'category_name',
    F.coalesce(F.split(F.col("category_code"),"\\.").getItem(0), F.lit('Others'))
)

In [0]:
# Top 5 products by revenue
revenue = result_df.filter(F.col('event_type')=='purchase')\
    .groupBy("product_id")\
    .agg(F.sum("price").alias('revenue'))\
    .orderBy(F.desc('revenue'))
revenue.limit(5).display()

In [0]:
# Top Brands in sales
result_df.filter(
    (F.col('event_type')=='purchase') & 
    (F.col('brand').isNotNull()))\
    .repartition("brand")\
    .groupBy("brand")\
    .agg(F.count("*").alias('most_purchased'))\
    .orderBy(F.desc('most_purchased')).limit(5).display()

In [0]:
# Running total per user
window = Window.partitionBy("user_id").orderBy("event_time")
full_df.select('user_id','event_time')\
    .withColumn("cumulative_events", F.count("*").over(window))\
    .display(10)



In [0]:
# Conversion rate by category
Category_conversion = result_df\
    .groupby('category_name')\
    .pivot("event_type", [ "view", "cart","purchase"])\
    .agg(F.count("category_name").alias('Count'))\
    .withColumn('cart_rate',F.round(F.col('cart')/F.col('view')*100,2))\
    .withColumn('purchase_rate',F.round(F.col('purchase')/F.col('view')*100,2))
Category_conversion.display()

In [0]:
# Joins
a = spark.createDataFrame([(1,"A"),(2,"B"),(3,"C")], ["user_id","val"])
b = spark.createDataFrame([(2,"X"),(3,"Y"),(4,"Z")], ["user_id","val2"])

In [0]:
inner = a.join(b,on='user_id',how='inner')
inner.show()

In [0]:
left = a.join(b,on='user_id',how='left')
left.show()

In [0]:
left_anti = a.join(b,on='user_id',how='left_anti')
left_anti.show()

In [0]:
left_outer = a.join(b,on='user_id',how='left_outer')
left_outer.show()

In [0]:
left_semi = a.join(b,on='user_id',how='left_semi')
left_semi.show()